# Notebook to reproduce 14 Her c NIRCam imaging with the 335R coronagraph

Author: W. Balmer (May 20th, 2025)

This notebook should allow the user to reproduce the data reduction, starlight subtraction, contrast calibration, and source extraction for JWST GO 3337 (DBG & Balmer et al. 2025)

# install pyklip, spaceklip, etc

To run this notebook, you'll need to create a new conda environment and install stpsf, pyklip, spaceklip, etc and then open this notebook with a ipykernel based in that environment with the correct environment variables.

Download the spaceklip git repo:
```
git clone https://github.com/spacetelescope/spaceKLIP.git
```
Create a new env with the right python version:
```
conda create -n spaceklip_v2p1 python=3.11
conda activate spaceklip_v2p1
```

(optionally, you can create a jupyter kernel, and run this notebook on that kernel explicitly):

```
pip install ipykernel
python -m ipykernel install --user --name=spaceklip_v2p1
```

Cd into the repo, checkout the version of spaceklip that was used for the paper (v2.1), install the requirements, and finally install spaceklip itself.
```
cd spaceKLIP
git checkout 11df3a1
pip install jwst==1.18
pip install -r requirements.txt
pip install -e .
```

In the paper, we used CRDS_VER='12.1.5  ' and the 'jwst_1364.pmap' CRDS context.

Download the stpsf reference files from [here](https://stpsf.readthedocs.io/en/latest/installation.html) and set up the correct environment variables, for example:
```
export STPSF_PATH=$HOME/data/stpsf-data
export WEBBPSF_EXT_PATH='$HOME/data/webbpsf_ext_data/'
export PYSYN_CDBS='$HOME/data/cdbs/'
```

In [ ]:
target='14Her'
pid=3337

# acquire data

To aquire nircam coron data, you can use the jwst_mast_query or spaceklip.mast api tools:

https://github.com/spacetelescope/jwst_mast_query

jwst_download.py -v --instrument nircam --propID 1193 -f uncal.fits --date_select 2022-01-01 2022-12-01

In [ ]:
from spaceKLIP import mast
import numpy as np

In [ ]:
data_dir = f'/Users/wbalmer/data/jwst/{target}/' # edit this to refer to your own file structure

In [ ]:
table = mast.query_coron_datasets('NIRCam', 
                                  'F444W', 
                                  'MASKA335R',
                                  program=pid,
                                  ignore_exclusive_access=False, 
                                  return_filenames=True, 
                                  level='uncal')

print(f"Found {len(table)} total rows.")
print(table[-5:-1])

In [ ]:
mast.download_files(table, outputdir=data_dir)

# initialize spaceklip and database object

In [ ]:
from __future__ import division

# =============================================================================
# IMPORTS
# =============================================================================

import os
import pdb
import sys

import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np

from spaceKLIP import database, coron1pipeline, coron2pipeline, coron3pipeline, pyklippipeline, imagetools, analysistools

# plotting
import matplotlib.pyplot as plt

In [ ]:
# Set the input and output directories and grab the input FITS files.

# are the data reduced yet?
reduced = False
# are the bad pixels corrected yet?
cleaned = False
# are the data aligned yet?
aligned = False

# extra options: defaults are those used in our paper
pad = False
coadd = False
crop = True
blur = True

data_dir = f'/Users/wbalmer/data/jwst/{target}/' # edit this to refer to your own file structure
output_dir = f'./spaceklip_{target}/' # edit this to refer to your own file structure

if aligned and coadd:
    input_dir = output_dir+'coadded/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif aligned:
    input_dir = output_dir+'aligned/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif cleaned:
    input_dir = output_dir+'medsub/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('calints.fits')])
elif reduced:
    input_dir = output_dir+'stage2/'
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])
else:
    input_dir = data_dir # pull uncal files from the directory jwst_mast_query downloads to (you might need to rename this)
    fitsfiles = sorted([input_dir + f for f in os.listdir(input_dir) if f.endswith('.fits')])

# spectral type of the star
spt = 'K0V' # 14 Her


# reductions

In [ ]:
# Initialize the spaceKLIP database and read the input FITS files.
Database = database.Database(output_dir=output_dir)
Database.read_jwst_s012_data(datapaths=fitsfiles,
                             psflibpaths=None,
                             bgpaths=None)

In [ ]:
Database.obs.keys()

In [ ]:
# To select a subset of obs you can downsize the dict as follows
# In the instructions to download the data via spaceklip.mast above, you'll only get the F444W 
# but using jwst_download you'll get both F200W, F444W, and some TAQ images. 

select_obs = [
              # 'JWST_NIRCAM_NRCA2_F200W_MASKRND_MASK335R_SUB320A335R',
              'JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R',
              ]

Database.obs = {k:Database.obs[k] for k in select_obs}

In [ ]:
if not reduced:
    coron1pipeline.run_obs(database=Database,
                           steps={'group_scale': {'skip': False},
                              'dq_init': {'skip': False},
                              'saturation': {'n_pix_grow_sat': 1,
                                             'grow_diagonal': False},
                              'ipc': {'skip': True},
                              'superbias':{'skip': False},
                              'refpix': {'odd_even_columns': True,
                                         'odd_even_rows': True,
                                         'nlower': 4,
                                         'nupper': 4,
                                         'nleft': 4,
                                         'nright': 4,
                                         'nrow_off': 0,
                                         'ncol_off': 0},
                              'linearity': {'skip': False},
                              'dark_current': {'skip': True},
                              'persistence': {'skip': True},
                              'jump': {'rejection_threshold': 4.,
                                       'three_group_rejection_threshold': 4.,
                                       'four_group_rejection_threshold': 4.,
                                       'maximum_cores': 'all'},
                              'subtract_1overf': {'model_type': 'median', # <--- this is not default
                                                  'sat_frac': 0.5,
                                                  'combine_ints': True,
                                                  'vertical_corr': True,
                                                  'nproc': 16
                                                 },
                              'ramp_fit': {
                                           'algorithm':'LIKELY', # <--- this is not default
                                           'save_calibrated_ramp': False,
                                           'maximum_cores': 'all'},
                              'gain_scale': {'skip': False}},
                           subdir='stage1')

In [ ]:
if not reduced:
    coron2pipeline.run_obs(database=Database,
                           steps={'outlier_detection': {'skip': False}},
                           subdir='stage2')

In [ ]:
ImageTools = imagetools.ImageTools(Database)

In [ ]:
if not cleaned:
    ImageTools.update_nircam_centers()

In [ ]:
if not cleaned:
    ImageTools.find_bad_pixels(method='dqarr+sigclip+timeints',
                               sigclip_kwargs={'sigclip': 4,
                                               'shift_x': [-1, 0, 1],
                                               'shift_y': [-1, 0, 1]},
                               timeints_kwargs={'sigma':4}
                              )
    ImageTools.clean_bad_pixels(method='timemed+interp2d',
                                interp2d_kwargs={'size':9}, # default 4
                                subdir='bpcleaned')

In [ ]:
if not cleaned:
    ImageTools.replace_nans(cval=0.,
                            types=['SCI', 'SCI_BG', 'REF', 'REF_BG'],
                            subdir='nanreplaced')

if not cleaned:
    ImageTools.subtract_median(types=['SCI', 'SCI_TA', 'SCI_BG', 'REF', 'REF_TA', 'REF_BG'],
                               subdir='medsub')

In [ ]:
if not cleaned and coadd:
    ImageTools.coadd_frames()

In [ ]:
if not aligned:
    ImageTools.recenter_frames(
        spectral_type=spt,
    )

    if blur:
        ImageTools.blur_frames(subdir='blur')

    ImageTools.align_frames(
        subdir='aligned',
    )
    

In [ ]:
if crop:
    ImageTools.crop_frames(npix=60) # from 320 -> 200

if pad:
    ImageTools.pad_frames(
                          npix=50,
                          types=['SCI', 'SCI_BG', 
                                 'REF', 'REF_BG'],
                          cval=0.
                         )

In [ ]:
maxnumbasis = Database.obs['JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R']['NINTS'].sum()

In [ ]:
numbasis = [1, 2, 3, 4, 5, 6, 7, 8, 9] + list(np.arange(10,maxnumbasis,10))

# klip

In [ ]:
# this cell is only necessary if you have an apple silicon chip
# unfortunately, I have an apple silicon chip.
# once other image reduction steps are done, run this so that pyklip.parallelized doesn't break
# see discussion here: https://pyklip.readthedocs.io/en/latest/install.html#note-on-parallelized-performance
import platform
if platform.processor() == 'arm':
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["OMP_NUM_THREADS"] = "1" 

In [ ]:
pyklippipeline.run_obs(database=Database,
                       kwargs={'mode': ['ADI+RDI'],
                               'annuli': [1],
                               'subsections': [1],
                               'numbasis':numbasis,
                               'algo': 'klip'},
                       subdir='klipsub')

In [ ]:
data = fits.getdata(Database.red['JWST_NIRCAM_NRCALONG_F444W_MASKRND_MASK335R_SUB320A335R']['FITSFILE'][-1])

In [ ]:
# display the starlight subtracted image:
plt.figure(figsize=(6,6))
plt.imshow(data[-1], origin='lower', cmap='magma')
plt.colorbar()

# analysis

In [ ]:
Database = database.Database(output_dir=output_dir)
fitsfiles = sorted([output_dir+'klipsub/' + f for f in os.listdir(output_dir+'klipsub/') if f.endswith('KLmodes-all.fits')])
Database.read_jwst_s3_data(fitsfiles)

In [ ]:
# compute raw contrast curve?
raw_contrasts = True
# calibrate said curve?
calib_contrasts = True
# extract sources?
comp_fm = True

In [ ]:
Analysis = analysistools.AnalysisTools(Database)

companions = [
    [-0.806, -0.758, 1e-6] # 14 Her c
             ] # units arcsecond, arcsecond, contrast

companion_masks = [
    [-0.806, -0.758, 1.5], [0.95, -1.25, 3.5] # 14 Her c and bg galaxy
] # units arcsecond, arcsecond, contrast

if companions == []:
    print('no comps')
    companions = None

if companion_masks == []:
    print('no comps masks')
    companion_masks = None


starfile = './14HERA_BTNEXTGEN_SCALED_MU_JY.txt'

In [ ]:
if raw_contrasts:
    Analysis.raw_contrast(starfile,spectral_type=spt,companions=companion_masks, plot_xlim=(0,3))

In [ ]:
inj_seps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.5, 3.0]
if calib_contrasts:
    if companion_masks is not None:
        Analysis.calibrate_contrast(
                                    companions=companion_masks,
                                    injection_seps=inj_seps,
                                    plot_xlim=(0,3)
                                   )
    else:
        Analysis.calibrate_contrast(injection_seps=inj_seps,
                                    plot_xlim=(0,3)
                                   )

In [ ]:
if comp_fm:
    if companions is not None:
        Analysis.extract_companions(companions, 
                                    starfile,
                                    0.0,
                                    spectral_type=spt,
                                    planetfile='./14HER_PLANET_LACY23_SCALED_MU_JY.txt',
                                    fitmethod='mcmc',
                                    fitkernel='matern32',
                                    nthreads=16,
                                    subtract=True,
                                    save_preklip=False,
                                   )